# 1.5 「ファインチューニング」で精度向上を実現する方法

- 本ファイルでは、学習済みのVGGモデルを使用し、ファインチューニングでアリとハチの画像を分類するモデルを学習します



# 学習目標

1.	PyTorchでGPUを使用する実装コードを書けるようになる
2.	最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる
3.	学習したネットワークを保存・ロードできるようになる



# 事前準備

- 1.4節で解説したAWS EC2 のGPUインスタンスを使用します


In [4]:
# パッケージのimport
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

In [5]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [22]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

In [32]:
OUTPUT_FEATURES = 2

LEARNING_RATE = 0.1
MOMENTUM = 0.9

NUM_EPOCHS = 2

# DatasetとDataLoaderを作成

In [24]:
# 1.3節で作成したクラスを同じフォルダにあるmake_dataset_dataloader.pyに記載して使用
from utils.dataloader_image_classification import (
    HymenopteraDataset,
    ImageTransform,
    make_datapath_list,
)

# アリとハチの画像へのファイルパスのリストを作成する
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

# Datasetを作成する
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase="train"
)

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase="val"
)


# DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


# モデルを学習させる関数を作成

In [31]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch + 1, num_epochs))
        print("-------------")

        # epochごとの訓練と検証のループ
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()  # モデルを訓練モードに
            else:
                net.eval()  # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == "train"):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

# 損失関数を定義

In [25]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

# ネットワークモデルの作成

In [26]:
vgg_models = {
    "vgg16": (models.vgg16, models.VGG16_Weights.DEFAULT),
    "vgg16_bn": (models.vgg16_bn, models.VGG16_BN_Weights.DEFAULT),
    "vgg19_bn": (models.vgg19, models.VGG19_Weights.DEFAULT),
    "vgg19_bn": (models.vgg19_bn, models.VGG19_BN_Weights.DEFAULT),
}

In [33]:
use_pretrained = True

for name, (model, weights) in vgg_models.items():
    _net = model(weights=weights)
    num_classifier_layers = len(_net.classifier)
    last_in_features = _net.classifier[num_classifier_layers - 1].in_features
    _net.classifier[num_classifier_layers - 1] = nn.Linear(
        in_features=last_in_features, out_features=OUTPUT_FEATURES
    )
    _net.train()
    print(name, num_classifier_layers, last_in_featuresatures)

    # 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
    for name, param in _net.named_parameters():
        param.requires_grad = False

    params_to_update = []
    for name, param in _net.classifier[num_classifier_layers - 1].named_parameters():
        param.requires_grad = True
        params_to_update.append(param)

    optimizer = optim.SGD(
        params_to_update,
        lr=LEARNING_RATE,
        momentum=MOMENTUM,
    )

    train_model(_net, dataloaders_dict, criterion, optimizer, num_epochs=NUM_EPOCHS)

weight
bias
-----------
[Parameter containing:
tensor([[ 0.0085,  0.0117,  0.0149,  ...,  0.0094,  0.0092, -0.0028],
        [-0.0146, -0.0013,  0.0002,  ..., -0.0089,  0.0073, -0.0025]],
       requires_grad=True), Parameter containing:
tensor([-0.0001,  0.0074], requires_grad=True)]
使用デバイス： cpu
Epoch 1/2
-------------


100%|██████████| 5/5 [01:36<00:00, 19.22s/it]


val Loss: 0.8600 Acc: 0.4248
Epoch 2/2
-------------


100%|██████████| 8/8 [02:12<00:00, 16.61s/it]


train Loss: 4.5838 Acc: 0.7860


...

val Loss: 0.6817 Acc: 0.5359
Epoch 2/2
-------------


100%|██████████| 8/8 [03:54<00:00, 29.36s/it]


train Loss: 0.9936 Acc: 0.7737


100%|██████████| 5/5 [02:31<00:00, 30.35s/it]

val Loss: 1.0786 Acc: 0.9477


# 学習したネットワークを保存・ロード

In [10]:
# PyTorchのネットワークパラメータの保存
save_path = "./weights_fine_tuning.pth"
torch.save(net.state_dict(), save_path)

In [11]:
# PyTorchのネットワークパラメータのロード
load_path = "./weights_fine_tuning.pth"
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={"cuda:0": "cpu"})
net.load_state_dict(load_weights)

<All keys matched successfully>

以上